# Empirical propagation on a toy example

In [ ]:
In this notebook we present how our empirical propagation approach works on a toy example.

In [ ]:
include("../src/DSI.jl")
include("../src/Zono_utils.jl")
include("../src/PZono.jl")
include("../src/DSZ.jl")
include("../src/propagation.jl")

In [ ]:
using PyPlot

We define the weights and biases for our toy network:

In [ ]:
#Toy network allowing us for Lipschitz approximation 
W1 = [1.0 -1.0; 1.0 1.0; -1.0 2.0]
b1 = [0.0; 0.0; 0.0]
W2 = [1.0 -1.0 1.0; 1.0 -1.0 2.0]
b2 = [0.0; 0.0]

L1 = Layer(W1, b1, ReLU())
L2 = Layer(W2, b2, Id())
full_net = Network([L1;L2])

x = [normal(interval(0,1),1), normal(interval(0,1),1)]

In [ ]:
nb_discretization_steps = 100
ProbabilityBoundsAnalysis.setSteps(nb_discretization_steps)
pz = pbox_approximate_nnet(full_net,x,true) 
if print_figures
    ProbabilityBoundsAnalysis.plot(pz[1])
    PyPlot.savefig("../pictures/Toy2Output1.png")
    ProbabilityBoundsAnalysis.plot(pz[2])
    PyPlot.savefig("../pictures/Toy2Output2.png")
end

Using Lipschitz verifier, we obtain that this network has a Lipschitz constant of 5 in the L2 norm. Recall that the Lipschitz constant in L1 norm will be at most $5\sqrt{2} = 7.071$. We combinatorically generate a $\varepsilon$-covering of the input, assuming the independence of inputs. 

Description of the input p-boxes:

In [ ]:
x_desc = [([0,1],1),([0,1],1)]

Generate samples:

In [ ]:
psamples = gaussian_samples(x_desc, 1000, 0.005)

We generated a covering with $\varepsilon = 0.005$ and $1000$ samples for each input within the covering. The psamples vector contains all of the samples. The first dimension of the vector represents the number of inputs of the neural network (for each input, we have a covering). The second dimension represents the cdfs within one pbox, and the third the number of samples for each cdf.

In [ ]:
println(size(psamples))
println(size(psamples[1]))
println(size(psamples[1][1]))

Propagate samples:

In [ ]:
outps = propagate_gaussians_multhread(psamples, full_net, 1000)

Visualise outputs:

In [ ]:
num_samples=1000
for i in 1:length(outps[1][1])
    ProbabilityBoundsAnalysis.plot(pz[i])
    for j in 1:length(outps)
        data = [outps[j][k][i] for k in 1:num_samples]
        cdf = empirical_cdf(data)
        plot(cdf[1], cdf[2], marker="o", linestyle="-")
    end
    filename = string("../pictures/empirical_cdfs_propagation_","full_net","_", i, ".png")
    PyPlot.savefig(filename)    
end